In [16]:
from board_labeler import get_files_in_dir

filesDir = "labeledConstSize/"

paths = get_files_in_dir(filesDir)

In [17]:
import cv2
import re
import os
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()

stringLen = len(filesDir)
m = re.compile(r"(\/)([a-zA-Z_]+)(\/)")

X = [cv2.imread(x) for x in paths]
y = [m.findall(x)[0][1] for x in paths]

dictClasses = {
    "empty":0,
    "bishop_black":1,
    "bishop_white":2,
    "king_black":3,
    "king_white":4,
    "knight_black":5,
    "knight_white":6,
    "pawn_black":7,
    "pawn_white":8,
    "queen_black":9,
    "queen_white":10,
    "rook_black":11,
    "rook_white":12,
}
y = [[dictClasses[x]] for x in y]

# temp1 = [x.shape for x in X]
# temp = set(temp1)

# temp3 = [temp1.count(x) for x in temp]
# print(temp)
# print(temp3)

print(f"{len(X)} {len(y)}")

1804 1804


In [18]:
from sklearn.model_selection import train_test_split
import numpy as np

X = np.asanyarray(X)
y = np.asanyarray(y)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [19]:
import shutil

train_path = 'train_data/'
test_path = 'test_data/'

if os.path.exists(train_path) and os.path.isdir(train_path):
    shutil.rmtree(train_path)
os.mkdir(train_path)

if os.path.exists(test_path) and os.path.isdir(test_path):
    shutil.rmtree(test_path)
os.mkdir(test_path)

for key, value in dictClasses.items():
    os.mkdir(f"{train_path}[{value}]")
    os.mkdir(f"{test_path}[{value}]")

for i in range(len(X_train)):
    cv2.imwrite(f"{train_path}{y_train[i]}/{i}.png", X_train[i])
    
for i in range(len(X_test)):
    cv2.imwrite(f"{test_path}{y_test[i]}/{i}.png", X_test[i])

In [20]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from board_finder import squareImgSize

datagen = ImageDataGenerator(rotation_range=90, width_shift_range=0.2, height_shift_range=0.2, brightness_range=(0.2, 0.8), horizontal_flip=True, vertical_flip=True)
generator = datagen.flow_from_directory(train_path, target_size=(squareImgSize, squareImgSize), batch_size=32, class_mode="sparse", seed=42)

Found 1208 images belonging to 13 classes.


In [21]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers


data_augmentation = keras.Sequential(
    [
        layers.RandomFlip("horizontal"),
        layers.RandomRotation(0.1),
    ]
)

imgShape = X_train[0].shape
num_classes = len(dictClasses)
print(num_classes)

inner_model = tf.keras.applications.resnet50.ResNet50(
    include_top=False,
    weights='imagenet',
    input_shape=X_train[0].shape,
    classes=num_classes,
)
inner_model.trainable = False
model = keras.Sequential([
    keras.Input(shape=X_train[0].shape),
    inner_model,
    layers.Flatten(),
    tf.keras.layers.Dense(1024, activation='relu'),
    tf.keras.layers.Dense(512, activation='relu'),
    #Dense(50, activation='relu'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(num_classes, activation='softmax'),
])

model.compile(
    optimizer='adam',   
    loss='categorical_crossentropy',
    metrics=['accuracy'],
)

model.fit(
    # train_generator,
    X_train,
    keras.utils.to_categorical(y_train),
    epochs=5,
    batch_size=128
)

13
Epoch 1/5
10/10 [==============================] - 12s 447ms/step - loss: 1.2761 - accuracy: 0.7219
Epoch 2/5
10/10 [==============================] - 4s 400ms/step - loss: 0.1242 - accuracy: 0.9644
Epoch 3/5
10/10 [==============================] - 4s 378ms/step - loss: 0.0305 - accuracy: 0.9942
Epoch 4/5
10/10 [==============================] - 4s 395ms/step - loss: 0.0104 - accuracy: 0.9983
Epoch 5/5
10/10 [==============================] - 4s 389ms/step - loss: 0.0054 - accuracy: 0.9983


In [22]:
score = model.evaluate(X_test, keras.utils.to_categorical(y_test), verbose = 0) 

print('Test loss:', score[0]) 
print('Test accuracy:', score[1])

Test loss: 0.005301191005855799
Test accuracy: 0.9966443181037903


In [23]:
model.save("models/secondmodel")

INFO:tensorflow:Assets written to: models/secondmodel\assets


C:\Users\leon.muller\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\keras\utils\generic_utils.py:494: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  warnings.warn('Custom mask layers require a config and must override '
